In [7]:
import andes
import pandapower as pp
from jams import dcopf

from andes.interop.pandapower import build_group_table, add_gencost

import numpy as np
import pandas as pd

In [8]:
print(andes.__version__)
print(pp.__version__)

1.6.5.post31.dev0+g5ec55493
2.8.0


Only linear cost or only quadratic cost, the results are the same.

In [9]:
def bc_dc(case_path, case_name):
    case = case_path + case_name
    ssa = andes.load(case,
                    setup=True,
                    no_output=True,
                    default_config=False)
    build_group_table(ssa, 'StaticGen', ['u', 'idx', 'p0'])

    ssd = dcopf()
    ssd.from_andes(ssa)
    ssd.cost.c1 = range(1,ssd.gen.shape[0]+1)
    ssd.cost.c2 = 0
    ssd.build()
    ssd.mdl.optimize()

    from andes.interop.pandapower import to_pandapower
    ssp = to_pandapower(ssa)

    # add gen cost
    gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
    gen_cost[:, 4] = ssd.cost.c2  # c2
    gen_cost[:, 5] = ssd.cost.c1  # c1

    add_gencost(ssp, gen_cost)
    pp.rundcopp(ssp)
    res_pp = ssp.res_gen
    res_jm = ssd.get_res()
    res_comp = pd.concat([res_pp, res_jm], axis=1)
    res_comp.p_sch = res_comp.p_sch * ssd.mva
    return res_comp.round(4)

case_path = '/Users/jinningwang/Documents/work/andes/jwang/jams/cases/'
case5 = 'pjm5bus.xlsx'
case14 = 'case14.xlsx'
case39 = 'case39.xlsx'
res5 = bc_dc(case_path, case5)
res14 = bc_dc(case_path, case14)
res39 = bc_dc(case_path, case39)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xb10f3f1e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+01, 1e+01]
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.000000000e+00
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0xba743882
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [

gen vm_pu > bus max_vm_pu for gens [2 3]. Setting bus limit for these gens.


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x95f18cee
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [5e+00, 1e+01]
  RHS range        [6e+01, 6e+01]
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.2542300e+01   2.607115e+01   0.000000e+00      0s
       1    2.9960070e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.996007000e+02


gen vm_pu > bus max_vm_pu for gens [5]. Setting bus limit for these gens.


In [10]:
res5

,p_mw,q_mvar,va_degree,vm_pu,gen,p_sch
0,1300.0,46.9205,5.6420,1.0,0,1300.0
1,-100.0,171.0009,-1.3438,1.0,2,-100.0
2,-100.0,-39.4140,4.3401,1.0,4,-100.0
3,-100.0,181.1884,0.0000,1.0,3,-100.0


In [11]:
res14

,p_mw,q_mvar,va_degree,vm_pu,gen,p_sch
0,140.0,43.5571,0.7651,1.0,2,140.0
1,100.0,25.0753,-1.1962,1.0,3,100.0
2,19.0,12.7309,-6.6211,1.0,4,19.0
3,0.0,17.6235,-6.4877,1.0,5,0.0
4,0.0,-16.5493,0.0000,1.0,1,0.0


In [12]:
res39

,p_mw,q_mvar,va_degree,vm_pu,gen,p_sch
0,1040.00,161.7616,24.2665,1.0,1,1040.00
1,725.00,206.9649,13.3613,1.0,3,725.00
2,652.00,108.2928,19.3832,1.0,4,652.00
3,508.00,166.6884,17.5519,1.0,5,508.00
4,687.00,210.6614,22.1292,1.0,6,687.00
5,580.00,100.1648,25.1579,1.0,7,580.00
6,564.00,-1.3694,21.9255,1.0,8,564.00
7,865.00,21.7327,27.3544,1.0,9,865.00
8,633.23,78.4674,-4.7612,1.0,10,633.23
9,0.00,221.5745,0.0000,1.0,2,0.00


In [13]:
case_path = '/Users/jinningwang/Documents/work/andes/jwang/jams/cases/'
case14 = 'case14.xlsx'

case = case_path + case14
ssa = andes.load(case,
                setup=True,
                no_output=True,
                default_config=False)
build_group_table(ssa, 'StaticGen', ['u', 'idx', 'p0'])

ssd = dcopf()
ssd.from_andes(ssa)
ssd.cost.c1 = range(1,ssd.gen.shape[0]+1)
ssd.cost.c2 = 0
ssd.build()
ssd.mdl.optimize()

from andes.interop.pandapower import to_pandapower
ssp = to_pandapower(ssa)

# add gen cost
gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
gen_cost[:, 4] = ssd.cost.c2  # c2
gen_cost[:, 5] = ssd.cost.c1  # c1

add_gencost(ssp, gen_cost)
pp.rundcopp(ssp)
res_pp = ssp.res_gen
res_jm = ssd.get_res()
res_comp = pd.concat([res_pp, res_jm], axis=1)
res_comp.p_sch = res_comp.p_sch * ssd.mva

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0xba743882
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 3e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5900000e+00   1.190000e+00   0.000000e+00      0s
       1    3.9700000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.970000000e+00


gen vm_pu > bus max_vm_pu for gens [2 3]. Setting bus limit for these gens.


In [15]:
ssd.get_res()

,gen,p_sch
0,2,1.40
1,3,1.00
2,4,0.19
3,5,0.00
4,1,0.00


In [14]:
ssd.cost

,idx,c2,c1,c0,cr
0,2,0,1,0,0
1,3,0,2,0,0
2,4,0,3,0,0
3,5,0,4,0,0
4,1,0,5,0,0
